In [1]:
import sqlite3
import json
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain import SQLDatabase
from dotenv import load_dotenv, find_dotenv
import sys
import stat
from langchain_community.llms import HuggingFaceEndpoint
from langchain_anthropic import ChatAnthropic

c:\Users\tomce\miniconda3\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Load up API Keys

In [2]:
load_dotenv(find_dotenv())
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

#### Create Database File for Monitors

In [3]:
# Function to create database and tables
def create_database():
    conn = sqlite3.connect('product_database_monitors.db')
    c = conn.cursor()

    # Create products table
    c.execute('''CREATE TABLE IF NOT EXISTS monitors (
                    id INTEGER PRIMARY KEY,
                    url TEXT,
                    title TEXT,
                    warranty INTEGER,
                    regular_price REAL,
                    happy_price REAL,
                    category TEXT
                 )''')

    # Create feature_products table
    c.execute('''CREATE TABLE IF NOT EXISTS feature_monitors (
                    id INTEGER PRIMARY KEY,
                    feature_name TEXT,
                    feature_value TEXT
                 )''')

    # Create product_feature table
    c.execute('''CREATE TABLE IF NOT EXISTS monitor_feature (
                    product_id INTEGER,
                    feature_id INTEGER,
                    FOREIGN KEY (product_id) REFERENCES monitors(id),
                    FOREIGN KEY (feature_id) REFERENCES feature_monitors(id)
                 )''')

    conn.commit()
    conn.close()

# Function to insert product data into database
def insert_product_data(product_data):
    conn = sqlite3.connect('product_database_monitors.db')
    c = conn.cursor()

    # Insert product data into products table
    c.execute('''INSERT INTO monitors (url, title, warranty, regular_price, happy_price, category)
                 VALUES (?, ?, ?, ?, ?, ?)''', (
                    product_data.get('url'),
                    product_data.get('title'),
                    product_data.get('warranty_months'),
                    product_data.get('regular_price_mkd'),
                    product_data.get('happy_price_mkd'),
                    product_data.get('category')
                 ))
    product_id = c.lastrowid

    # Insert feature data into feature_gaming_laptops table and product_feature table
    for feature, value in product_data.get('features', {}).items():
        # Check if feature already exists
        c.execute('''SELECT id FROM feature_monitors WHERE feature_name = ? AND feature_value = ?''', (feature, value))
        feature_row = c.fetchone()
        if feature_row:
            feature_id = feature_row[0]
        else:
            c.execute('''INSERT INTO feature_monitors (feature_name, feature_value) VALUES (?, ?)''', (feature, value))
            feature_id = c.lastrowid

        # Insert into product_feature table
        c.execute('''INSERT INTO monitor_feature (product_id, feature_id) VALUES (?, ?)''',
                  (product_id, feature_id))

    conn.commit()
    conn.close()

# Function to read JSON files from a directory and insert data into database
def process_json_files(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            with open(os.path.join(directory, filename), 'r') as file:
                product_data = json.load(file)
                insert_product_data(product_data)

# Create database and tables
create_database()

# Process JSON files and insert data into database
json_directory = r'C:\Users\tomce\OneDrive - UKIM, FINKI\Desktop\Fakultet 3ta Godina\2 Sesti Semestar\0 DATA SCIENCE SEMINARSKA\1 Starting Over\products_categories_fixed\monitors\monitors_new_template'
process_json_files(json_directory)


#### Get All "feature_name=feature_value" pairs for each key-value 

In [ ]:
def get_features_string(db_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Query to select all features
    cursor.execute("SELECT feature_name, feature_value FROM feature_monitors WHERE feature_value IS NOT NULL")
    
    # Fetch all rows from the executed query
    features = cursor.fetchall()
    
    # Close the database connection
    conn.close()
    
    # Format the features into the desired string format
    features_string = ", ".join([f"{name}={value}" for name, value in features])
    
    return features_string

# Example usage
db_path = 'product_database_monitors.db'
features_string = get_features_string(db_path)
print(features_string)

#### Connect with Database File

In [5]:
conn = sqlite3.connect('product_database_monitors.db')
c = conn.cursor()
input_db = SQLDatabase.from_uri('sqlite:///product_database_monitors.db')

#### Chat Prompt Template for Communicating With Monitors

In [8]:
template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Please note that in the schema, the regular_price is the normal price, while the happy_price is a discounted one.
Table Descriptions:
monitors Table:
id: An integer representing the primary key.
url: A text field for storing URLs.
title: A text field for the product title.
warranty: An integer field for warranty information.
regular_price: A real number field for the regular price.
happy_price: A real number field for the discounted price.
category: A text field for product category.

feature_monitors Table:
id: An integer representing the primary key.
feature_name: A text field for feature names (e.g., resolution, refresh rate).
feature_value: A text field for feature values (e.g., 1920x1080, 60Hz).

monitor_feature Table:
product_id: An integer representing the product ID (foreign key referencing the monitors table).
feature_id: An integer representing the feature ID (foreign key referencing the feature_monitors table)."

Example input for questions asking for multiple features: 'What OLED TVs are there that have a display bigger than 60 inches?'
Example output for questions asking for multiple features:
'SELECT tvs.title, screen_size.feature_value, screen_type.feature_value 
FROM tvs
JOIN tv_feature AS tvf1 ON tvs.id = tvf1.product_id
JOIN feature_tvs AS screen_size ON tvf1.feature_id = screen_size.id
JOIN tv_feature AS tvf2 ON tvs.id = tvf2.product_id
JOIN feature_tvs AS screen_type ON tvf2.feature_id = screen_type.id
WHERE screen_size.feature_name = 'screen_size_inches'
AND CAST(screen_size.feature_value AS INTEGER) >= 60
AND screen_type.feature_name = 'panel_type'
AND screen_type.feature_value LIKE '%OLED%';'
Don't include a feature in the query if the user doesn't ask for it.

You ABSOLUTELY MUST use the ‘LIKE’ operator instead of ‘=’ in the SQL query for columns in the tables that are of TEXT value.
You ABSOLUTELY MUST use the ‘=’ operator instead of ‘LIKE’ in the SQL query for columns in the tables that are of REAL, DOUBLE, or INTEGER value.
Here are all the features a user might ask for:
{features}
Question: {question}
SQL Query:
"""
prompt = ChatPromptTemplate.from_template(template)

def get_schema(db):
    schema = input_db.get_table_info()
    return schema

#### 3 Models to Choose From: HuggingFace Mistral, Gemini, and Claude Sonnet

In [9]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.3"
llm = HuggingFaceEndpoint(
    repo_id=repo_id, temperature=0.1, token=HUGGINGFACEHUB_API_TOKEN
)

WARNING! token is not default parameter.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\tomce\.cache\huggingface\token
Login successful


In [19]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.5)

In [27]:
llm = ChatAnthropic(temperature=0, model_name="claude-3-sonnet-20240229")

#### Ask Question to Get Query Answer

In [11]:
sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

user_question = 'What monitors are there that have a refresh rate of above 60Hz and have a flat screen?'
smth= sql_chain.invoke({"features": features_string,"question": user_question})
print(smth)
output_string = smth.replace("```sql", "").replace("```", "")
print(output_string)

```
SELECT monitors.title, feature_monitors.feature_name, feature_monitors.feature_value
FROM monitors
JOIN monitor_feature ON monitors.id = monitor_feature.product_id
JOIN feature_monitors ON monitor_feature.feature_id = feature_monitors.id
WHERE feature_monitors.feature_name IN ('screen_refresh_rate_hz', 'screen_type')
AND feature_monitors.feature_value > 60
AND feature_monitors.feature_value LIKE '%Flat%';
```
This query will return the title, feature name, and feature value for all monitors that have a refresh rate above 60Hz and have a flat screen.

SELECT monitors.title, feature_monitors.feature_name, feature_monitors.feature_value
FROM monitors
JOIN monitor_feature ON monitors.id = monitor_feature.product_id
JOIN feature_monitors ON monitor_feature.feature_id = feature_monitors.id
WHERE feature_monitors.feature_name IN ('screen_refresh_rate_hz', 'screen_type')
AND feature_monitors.feature_value > 60
AND feature_monitors.feature_value LIKE '%Flat%';

This query will return the ti

In [10]:
sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

user_question = 'What monitors are there that have a refresh rate of above 60Hz?'
smth= sql_chain.invoke({"features": features_string,"question": user_question})
print(smth)
output_string = smth.replace("```sql", "").replace("```", "")
print(output_string)

```
SELECT monitors.title, feature_monitors.feature_name, feature_monitors.feature_value
FROM monitors
JOIN monitor_feature ON monitors.id = monitor_feature.product_id
JOIN feature_monitors ON monitor_feature.feature_id = feature_monitors.id
WHERE feature_monitors.feature_name = 'screen_refresh_rate_hz'
AND CAST(feature_monitors.feature_value AS INTEGER) > 60;
```
This query will return the title, feature name, and feature value for all monitors that have a refresh rate above 60Hz.

SELECT monitors.title, feature_monitors.feature_name, feature_monitors.feature_value
FROM monitors
JOIN monitor_feature ON monitors.id = monitor_feature.product_id
JOIN feature_monitors ON monitor_feature.feature_id = feature_monitors.id
WHERE feature_monitors.feature_name = 'screen_refresh_rate_hz'
AND CAST(feature_monitors.feature_value AS INTEGER) > 60;

This query will return the title, feature name, and feature value for all monitors that have a refresh rate above 60Hz.
